# Azure AI Services Demo - インフラストラクチャデプロイメント

このNotebookでは、Azure AI Services Demoプロジェクトの**インフラストラクチャを段階的にデプロイ**します。

実行前に Azure CLI にログインし、正しいサブスクリプションを選択してください。

## 1. 環境設定と前提条件の確認

In [106]:
# 設定: 必要に応じてここを書き換えてください
import os
ENVIRONMENT = os.environ.get('AZAI_ENV', 'dev')
RESOURCE_GROUP_NAME = os.environ.get('AZAI_RG', 'rg-aiservices-demo')
LOCATION = os.environ.get('AZAI_LOCATION', 'eastus')
DEPLOYMENT_NAME = os.environ.get('AZAI_DEPLOYMENT', f'ai-deploy-{ENVIRONMENT}')
PROJECT_PREFIX = os.environ.get('AZAI_PREFIX', 'aiproj')

print(f"ENVIRONMENT={ENVIRONMENT}")
print(f"RESOURCE_GROUP_NAME={RESOURCE_GROUP_NAME}")
print(f"LOCATION={LOCATION}")
print(f"DEPLOYMENT_NAME={DEPLOYMENT_NAME}")
print(f"PROJECT_PREFIX={PROJECT_PREFIX}")

ENVIRONMENT=dev
RESOURCE_GROUP_NAME=rg-aiservices-demo
LOCATION=eastus
DEPLOYMENT_NAME=ai-deploy-dev
PROJECT_PREFIX=aiproj


In [107]:
# 必要なモジュールをインポート
import subprocess
import json
import os
import time
from datetime import datetime

def run_command(command, description=""):
    """実行補助: shell コマンドを実行して stdout/stderr を返す。
    戻り値は stdout (成功) または None (失敗) です。
    この実装はサブプロセスの出力をバイトで受け取り、UTF-8にデコードできないバイトが
    含まれていても errors='replace' で置換して安全に処理します。"""
    if description:
        print(f"🔄 {description}")
    try:
        # バイトで取得してから明示的にデコードすることで、
        # スレッド内でのデコード例外を防ぎます
        result = subprocess.run(command, shell=True, capture_output=True)
    except Exception as e:
        print(f"❌ 例外発生 (subprocess起動): {str(e)}")
        return None

    # result.stdout/result.stderr は bytes (または None) です。安全にデコードします。
    try:
        stdout = result.stdout.decode('utf-8') if result.stdout is not None else ''
    except Exception:
        # デコードに失敗したら置換モードで再デコードして生データを保つ
        try:
            stdout = result.stdout.decode('utf-8', errors='replace') if result.stdout is not None else ''
        except Exception:
            stdout = str(result.stdout)
    try:
        stderr = result.stderr.decode('utf-8') if result.stderr is not None else ''
    except Exception:
        try:
            stderr = result.stderr.decode('utf-8', errors='replace') if result.stderr is not None else ''
        except Exception:
            stderr = str(result.stderr)

    if result.returncode == 0:
        if description:
            print(f"✅ 成功: {description}")
        if stdout and stdout.strip():
            print(stdout)
        return stdout
    else:
        print(f"❌ エラー: {description} (コード={result.returncode})")
        if stderr and stderr.strip():
            print(stderr)
        return None

print("✅ ヘルパー関数が定義されました")

✅ ヘルパー関数が定義されました


## 2. Azure CLI とBicep の確認

In [108]:
# 対話的なサブスクリプション選択セル
def select_subscription_interactive():
    """
    利用可能なサブスクリプションを一覧表示し、対話的に選択して切り替えます。
    非対話環境では手動コマンドを案内します。
    """
    accounts_json = run_command("az account list --output json", "取得サブスクリプション情報")
    # run_command は成功時に stdout を文字列で返します。None や空白のみの出力を考慮して堅牢に処理します。
    if accounts_json is None:
        print("❌ サブスクリプション情報を取得できませんでした (コマンドが失敗しました)")
        return False
    accounts_json_str = str(accounts_json).strip()
    if accounts_json_str == '':
        print("❌ サブスクリプション情報が空でした。")
        print("👉 手動で確認: az account list --output json")
        return False
    try:
        accounts = json.loads(accounts_json_str)
    except Exception as e:
        print("❌ サブスクリプション情報の解析に失敗しました:", e)
        print("--- raw output (truncated) ---")
        print(accounts_json_str[:2000])
        print("--- end raw output ---")
        return False
    if not isinstance(accounts, list):
        print("❌ 取得したデータが期待する形式ではありません (リストではありません)")
        print(type(accounts))
        return False
    if len(accounts) == 0:
        print("❌ 利用可能なサブスクリプションがありません")
        return False
    elif len(accounts) == 1:
        print(f"✅ サブスクリプション '{accounts[0].get('name')}' を使用します")
        return True
    print('\n🔍 利用可能なサブスクリプション一覧:')
    for i, acc in enumerate(accounts, 1):
        marker = ' (現在)' if acc.get('isDefault') else ''
        print(f"{i}. {acc.get('name')} - {acc.get('id')}{marker}")
    try:
        choice = input('選択するサブスクリプションの番号を入力してください（Enterで現状維持）: ').strip()
    except Exception:
        print("⚠️ この環境では対話入力が利用できません。手動で以下のコマンドを実行してください:")
        print("   az account set --subscription <subscription-id-or-name>")
        return True
    if choice == '':
        print("✅ 現在のサブスクリプションをそのまま使用します")
        return True
    try:
        idx = int(choice)
        if idx < 1 or idx > len(accounts):
            print("❌ 無効な番号です")
            return False
        target = accounts[idx-1]
        sub_id = target.get('id') or target.get('name')
        # Use double quotes around the subscription value so cmd.exe / PowerShell do not pass literal single quotes
        res = run_command(f"az account set --subscription \"{sub_id}\"", f"サブスクリプションを {target.get('name')} に設定")
        return res is not None
    except ValueError:
        print("❌ 数字を入力してください")
        return False

# 実行
print("🔍 サブスクリプション選択を確認中...")
selection_ok = select_subscription_interactive()
print(f"🔔 選択結果: {selection_ok}")

🔍 サブスクリプション選択を確認中...
🔄 取得サブスクリプション情報
✅ 成功: 取得サブスクリプション情報
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "c02aa725-a3d5-4aa0-a954-f6a30d621628",
    "id": "233a6ea3-2df4-4489-baf1-9a84aab854ab",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Azure Pass - �X�|���T�[ �v����",
    "state": "Enabled",
    "tenantId": "c02aa725-a3d5-4aa0-a954-f6a30d621628",
    "user": {
      "name": "user3@mcw1218018outlook.onmicrosoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "16b3c013-d300-468d-ac64-7eda0820b6d3",
    "id": "7712a98a-1373-40e4-8c0f-a09ade38e5c3",
    "isDefault": false,
    "managedByTenants": [
      {
        "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47"
      },
      {
        "tenantId": "15f8080f-593b-49a3-8114-5b00631becf6"
      },
      {
        "tenantId": "cd266298-f166-4631-b917-4c3bdf4ef51d"
      },
      {
        "tenantId": "a1e1b00a-e2f5-4a82-a207-6f132038aa26"
      },
      {
        

## 3. Azure ログインとアカウント選択

In [109]:
# 最終的なアカウント情報確認
print("📋 最終的なアカウント設定を確認...")
final_account_info = run_command("az account show", "デプロイメント用アカウント確認")

if final_account_info:
    try:
        account_data = json.loads(final_account_info)
        print("\n✅ デプロイメントに使用するアカウント情報:")
        print("=" * 60)
        print(f"👤 ユーザー: {account_data.get('user', {}).get('name', 'N/A')}")
        print(f"📊 サブスクリプション: {account_data.get('name', 'N/A')}")
        print(f"🆔 サブスクリプションID: {account_data.get('id', 'N/A')}")
        print(f"🏢 テナント: {account_data.get('tenantId', 'N/A')}")
        print(f"🌍 環境: {account_data.get('environmentName', 'AzureCloud')}")
        
        # 権限の確認
        print(f"\n🔐 権限確認:")
        print(f"   - サブスクリプション状態: {account_data.get('state', 'N/A')}")
        
        if account_data.get('state', '').lower() == 'enabled':
            print("   ✅ サブスクリプションが有効です")
        else:
            print("   ⚠️ サブスクリプションの状態を確認してください")
            
        print("\n💡 この設定でデプロイメントを続行します")
        print("❌ 変更が必要な場合は、上記のサブスクリプション選択セルを再実行してください")
        
    except json.JSONDecodeError:
        print("❌ アカウント情報の解析に失敗しました")
        print("手動で確認してください: az account show")
else:
    print("❌ アカウント情報を取得できませんでした")
    print("再度ログインしてください: az login")

📋 最終的なアカウント設定を確認...
🔄 デプロイメント用アカウント確認
✅ 成功: デプロイメント用アカウント確認
{
  "environmentName": "AzureCloud",
  "homeTenantId": "549ce5b6-1653-4860-bd39-932e85df8aae",
  "id": "02bd8675-0a2c-4ff5-9087-2b3efcf57cd5",
  "isDefault": true,
  "managedByTenants": [
    {
      "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47"
    }
  ],
  "name": "ME-MngEnvMCAP039580-toohta-1",
  "state": "Enabled",
  "tenantId": "549ce5b6-1653-4860-bd39-932e85df8aae",
  "user": {
    "name": "toohta@MngEnvMCAP039580.onmicrosoft.com",
    "type": "user"
  }
}


✅ デプロイメントに使用するアカウント情報:
👤 ユーザー: toohta@MngEnvMCAP039580.onmicrosoft.com
📊 サブスクリプション: ME-MngEnvMCAP039580-toohta-1
🆔 サブスクリプションID: 02bd8675-0a2c-4ff5-9087-2b3efcf57cd5
🏢 テナント: 549ce5b6-1653-4860-bd39-932e85df8aae
🌍 環境: AzureCloud

🔐 権限確認:
   - サブスクリプション状態: Enabled
   ✅ サブスクリプションが有効です

💡 この設定でデプロイメントを続行します
❌ 変更が必要な場合は、上記のサブスクリプション選択セルを再実行してください
✅ 成功: デプロイメント用アカウント確認
{
  "environmentName": "AzureCloud",
  "homeTenantId": "549ce5b6-1653-4860-bd39-932e85df8aae",
  "id"

## 3. リソースグループの作成

In [110]:
# リソースグループの存在確認
# NOTE: 一部の静的解析ツールはノートブック内の別セルで定義された変数を検出できず、
# 直接 f-string 内で参照すると NameError を報告することがあります。
# そこで globals() と環境変数をフォールバックとして利用して安全に取得します。
rg_name = globals().get('RESOURCE_GROUP_NAME', os.environ.get('AZAI_RG', 'rg-aiservices-demo'))
location = globals().get('LOCATION', os.environ.get('AZAI_LOCATION', 'japaneast'))
rg_check = run_command(
    f"az group show --name {rg_name}",
    f"リソースグループ '{rg_name}' の存在確認"
)

if not rg_check:
    # リソースグループが存在しない場合は作成
    create_rg = run_command(
        f"az group create --name {rg_name} --location {location}",
        f"リソースグループ '{rg_name}' を作成中"
    )
else:
    print(f"✅ リソースグループ '{rg_name}' は既に存在します")

🔄 リソースグループ 'rg-aiservices-demo' の存在確認
✅ 成功: リソースグループ 'rg-aiservices-demo' の存在確認
{
  "id": "/subscriptions/02bd8675-0a2c-4ff5-9087-2b3efcf57cd5/resourceGroups/rg-aiservices-demo",
  "location": "eastus",
  "managedBy": null,
  "name": "rg-aiservices-demo",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}

✅ リソースグループ 'rg-aiservices-demo' は既に存在します
✅ 成功: リソースグループ 'rg-aiservices-demo' の存在確認
{
  "id": "/subscriptions/02bd8675-0a2c-4ff5-9087-2b3efcf57cd5/resourceGroups/rg-aiservices-demo",
  "location": "eastus",
  "managedBy": null,
  "name": "rg-aiservices-demo",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}

✅ リソースグループ 'rg-aiservices-demo' は既に存在します


## 4. Bicep テンプレートの検証

In [111]:
# Bicep ファイルのパスを確認
bicep_file = "main.bicep"
param_file = f"parameters/{ENVIRONMENT}.bicepparam"

print(f"📁 Bicepファイル: {bicep_file}")
print(f"📁 パラメータファイル: {param_file}")

# ファイル存在確認
if os.path.exists(bicep_file):
    print(f"✅ {bicep_file} が見つかりました")
else:
    print(f"❌ {bicep_file} が見つかりません")

if os.path.exists(param_file):
    print(f"✅ {param_file} が見つかりました")
else:
    print(f"❌ {param_file} が見つかりません")

📁 Bicepファイル: main.bicep
📁 パラメータファイル: parameters/dev.bicepparam
✅ main.bicep が見つかりました
✅ parameters/dev.bicepparam が見つかりました


In [ ]:
# Bicep テンプレートのコンパイル検証
compile_result = run_command(
    f"az bicep build --file {bicep_file}",
    "Bicepテンプレートのコンパイル検証"
)

# パラメータファイルの検証
param_compile = run_command(
    f"az bicep build-params --file {param_file}",
    "パラメータファイルのコンパイル検証"
)

## 5. デプロイメント What-If 検証

In [113]:
# What-If デプロイメント実行（実際のリソースは作成されない）
whatif_command = f"""az deployment group what-if \
--resource-group {RESOURCE_GROUP_NAME} \
--template-file {bicep_file} \
--parameters {param_file} \
--name {DEPLOYMENT_NAME}"""

print("🔍 What-If デプロイメントを実行中...")
print("📝 このステップでは実際のリソースは作成されません")
print("\n実行コマンド:")
print(whatif_command.replace(' \\', ' \\n'))
print("\n" + "="*50)

whatif_result = run_command(whatif_command, "What-If デプロイメント検証")

🔍 What-If デプロイメントを実行中...
📝 このステップでは実際のリソースは作成されません

実行コマンド:
az deployment group what-if --resource-group rg-aiservices-demo --template-file main.bicep --parameters parameters/dev.bicepparam --name ai-deploy-dev

🔄 What-If デプロイメント検証
✅ 成功: What-If デプロイメント検証
Note: The result may contain false positive predictions (noise).
You can help us improve the accuracy of the result by opening an issue here: https://aka.ms/WhatIfIssues

Resource and property changes are indicated with this symbol:
  + Create

The deployment will update the following scope:

Scope: /subscriptions/02bd8675-0a2c-4ff5-9087-2b3efcf57cd5/resourceGroups/rg-aiservices-demo

  + Microsoft.CognitiveServices/accounts/aiservicesdemo-ai-dev-2ape77 [2025-06-01]

      apiVersion:                               "2025-06-01"
      id:                                       "/subscriptions/02bd8675-0a2c-4ff5-9087-2b3efcf57cd5/resourceGroups/rg-aiservices-demo/providers/Microsoft.CognitiveServices/accounts/aiservicesdemo-ai-dev-2ape77"

## 6. 実際のデプロイメント実行

In [114]:
# ユーザー確認
print("⚠️  実際のデプロイメントを開始します")
print("📊 上記のWhat-If結果を確認してください")
print("💰 このデプロイメントではAzure料金が発生する可能性があります")
print("\n続行する場合は、次のセルを実行してください")

⚠️  実際のデプロイメントを開始します
📊 上記のWhat-If結果を確認してください
💰 このデプロイメントではAzure料金が発生する可能性があります

続行する場合は、次のセルを実行してください


In [115]:
# 実際のデプロイメント実行
deploy_command = f"""az deployment group create \
--resource-group {RESOURCE_GROUP_NAME} \
--template-file {bicep_file} \
--parameters {param_file} \
--name {DEPLOYMENT_NAME} \
--output table"""

print(f"🚀 デプロイメントを開始します: {DEPLOYMENT_NAME}")
print(f"⏰ 開始時刻: {datetime.now().strftime('%H:%M:%S')}")
print("\n📝 このプロセスには数分かかる場合があります...\n")

start_time = time.time()

deploy_result = run_command(deploy_command, "Azure AI Services インフラストラクチャのデプロイ")

end_time = time.time()
duration = int(end_time - start_time)

print(f"\n⏱️ デプロイメント時間: {duration // 60}分 {duration % 60}秒")
print(f"🕐 完了時刻: {datetime.now().strftime('%H:%M:%S')}")

🚀 デプロイメントを開始します: ai-deploy-dev
⏰ 開始時刻: 16:51:00

📝 このプロセスには数分かかる場合があります...

🔄 Azure AI Services インフラストラクチャのデプロイ
✅ 成功: Azure AI Services インフラストラクチャのデプロイ
Name           State      Timestamp                         Mode         ResourceGroup
-------------  ---------  --------------------------------  -----------  ------------------
ai-deploy-dev  Succeeded  2025-11-12T07:52:04.079368+00:00  Incremental  rg-aiservices-demo


⏱️ デプロイメント時間: 1分 31秒
🕐 完了時刻: 16:52:31
✅ 成功: Azure AI Services インフラストラクチャのデプロイ
Name           State      Timestamp                         Mode         ResourceGroup
-------------  ---------  --------------------------------  -----------  ------------------
ai-deploy-dev  Succeeded  2025-11-12T07:52:04.079368+00:00  Incremental  rg-aiservices-demo


⏱️ デプロイメント時間: 1分 31秒
🕐 完了時刻: 16:52:31


## 7. デプロイメント結果の確認

In [119]:
# デプロイメント出力値の取得
outputs_command = f"""az deployment group show \
--resource-group {RESOURCE_GROUP_NAME} \
--name {DEPLOYMENT_NAME} \
--query "properties.outputs" \
--output json"""

outputs_result = run_command(outputs_command, "デプロイメント出力値の取得")

if outputs_result:
    try:
        outputs_data = json.loads(outputs_result)
        
        print("\n📋 デプロイメント出力値:")
        print("="*50)
        
        for key, value in outputs_data.items():
            if key.endswith('Endpoint'):
                print(f"🔗 {key}: {value.get('value', 'N/A')}")
            elif key.endswith('Region'):
                print(f"🌏 {key}: {value.get('value', 'N/A')}")
            else:
                print(f"📄 {key}: [Object - 詳細は手動で確認]")
                
        # JSON形式でも出力（コピー用）
        print("\n📋 JSON形式（設定ファイル用）:")
        print(json.dumps(outputs_data, indent=2, ensure_ascii=False))
        
    except json.JSONDecodeError:
        print("❌ 出力値の解析に失敗しました")
        print(outputs_result)

🔄 デプロイメント出力値の取得
✅ 成功: デプロイメント出力値の取得
{
  "aiServicesEndpoint": {
    "type": "String",
    "value": "https://aiservicesdemo-ai-dev-2ape77.cognitiveservices.azure.com/"
  },
  "aiServicesEndpoints": {
    "type": "Object",
    "value": {
      "AI Foundry API": "https://aiservicesdemo-ai-dev-2ape77.services.ai.azure.com/",
      "Azure AI Model Inference API": "https://aiservicesdemo-ai-dev-2ape77.services.ai.azure.com/",
      "Azure OpenAI Legacy API - Latest moniker": "https://aiservicesdemo-ai-dev-2ape77.openai.azure.com/",
      "Cohere AI API": "https://aiservicesdemo-ai-dev-2ape77.services.ai.azure.com/",
      "Computer Vision": "https://aiservicesdemo-ai-dev-2ape77.cognitiveservices.azure.com/",
      "Container": "https://aiservicesdemo-ai-dev-2ape77.cognitiveservices.azure.com/",
      "Content Safety": "https://aiservicesdemo-ai-dev-2ape77.cognitiveservices.azure.com/",
      "Content Understanding": "https://aiservicesdemo-ai-dev-2ape77.services.ai.azure.com/",
      "Conver

In [120]:
# デプロイされたリソースの一覧表示
resources_command = f"az resource list --resource-group {RESOURCE_GROUP_NAME} --output table"

resources_result = run_command(resources_command, "デプロイされたリソース一覧")

print("\n📦 作成されたリソース一覧:")
print("="*70)

🔄 デプロイされたリソース一覧
✅ 成功: デプロイされたリソース一覧
Name                              ResourceGroup       Location    Type                                  Status
--------------------------------  ------------------  ----------  ------------------------------------  --------
aiservicesdemo-lang-dev-2ape77    rg-aiservices-demo  eastus      Microsoft.CognitiveServices/accounts
aiservicesdemo-cv-dev-2ape77      rg-aiservices-demo  eastus      Microsoft.CognitiveServices/accounts
aiservicesdemo-speech-dev-2ape77  rg-aiservices-demo  eastus      Microsoft.CognitiveServices/accounts
aiservicesdemo-ai-dev-2ape77      rg-aiservices-demo  eastus      Microsoft.CognitiveServices/accounts


📦 作成されたリソース一覧:
✅ 成功: デプロイされたリソース一覧
Name                              ResourceGroup       Location    Type                                  Status
--------------------------------  ------------------  ----------  ------------------------------------  --------
aiservicesdemo-lang-dev-2ape77    rg-aiservices-demo  eastus      M

## 8. 環境変数ファイルの生成

In [121]:
# 環境変数ファイル (.env) の生成
env_file_content = []
env_file_content.append("# Azure AI Services Demo Configuration")
env_file_content.append(f"# Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
env_file_content.append(f"# Environment: {ENVIRONMENT}")
env_file_content.append("")

if outputs_result:
    try:
        outputs_data = json.loads(outputs_result)
        
        # エンドポイント設定
        if 'aiServicesEndpoint' in outputs_data:
            env_file_content.append(f"AZURE_OPENAI_ENDPOINT={outputs_data['aiServicesEndpoint']['value']}")
            env_file_content.append("AZURE_OPENAI_API_VERSION=2024-02-15-preview")
            env_file_content.append("GPT4O_DEPLOYMENT_NAME=gpt-4o")
            env_file_content.append("GPT35_DEPLOYMENT_NAME=gpt-35-turbo")
            env_file_content.append("EMBEDDING_DEPLOYMENT_NAME=text-embedding-ada-002")
            # DALL-E (画像生成) 関連の初期設定を追加
            env_file_content.append("DALLE_DEPLOYMENT_NAME=dall-e-3")
            env_file_content.append("# DALLE_DEFAULT_SIZE=1024x1024")
            env_file_content.append("# DALLE_DEFAULT_QUALITY=standard")
            env_file_content.append("# DALLE_DEFAULT_STYLE=vivid")
            env_file_content.append("")
        
        if 'computerVisionEndpoint' in outputs_data:
            env_file_content.append(f"COMPUTER_VISION_ENDPOINT={outputs_data['computerVisionEndpoint']['value']}")
            env_file_content.append("")
        
        if 'speechServicesEndpoint' in outputs_data:
            env_file_content.append(f"SPEECH_SERVICES_ENDPOINT={outputs_data['speechServicesEndpoint']['value']}")
        
        if 'speechServicesRegion' in outputs_data:
            env_file_content.append(f"SPEECH_SERVICES_REGION={outputs_data['speechServicesRegion']['value']}")
            env_file_content.append("")
        
        if 'languageServicesEndpoint' in outputs_data:
            env_file_content.append(f"LANGUAGE_SERVICES_ENDPOINT={outputs_data['languageServicesEndpoint']['value']}")
            env_file_content.append("")
        
        # セキュリティメモ
        env_file_content.extend([
            "# セキュリティ上の理由により、APIキーは手動で設定してください:",
            "# AZURE_OPENAI_API_KEY=your_openai_api_key",
            "# COMPUTER_VISION_API_KEY=your_computer_vision_key",
            "# SPEECH_SERVICES_API_KEY=your_speech_services_key",
            "# LANGUAGE_SERVICES_API_KEY=your_language_services_key"
        ])
        
    except json.JSONDecodeError:
        env_file_content.append("# Error: デプロイメント出力の解析に失敗しました")

# .envファイルを保存
env_file_path = f"../.env.{ENVIRONMENT}"
with open(env_file_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(env_file_content))

print(f"✅ 環境変数ファイルを生成しました: {env_file_path}")
print("\n📄 ファイル内容:")
print("="*50)
with open(env_file_path, 'r', encoding='utf-8') as f:
    print(f.read())

✅ 環境変数ファイルを生成しました: ../.env.dev

📄 ファイル内容:
# Azure AI Services Demo Configuration
# Generated on 2025-11-12 17:11:56
# Environment: dev

AZURE_OPENAI_ENDPOINT=https://aiservicesdemo-ai-dev-2ape77.cognitiveservices.azure.com/
AZURE_OPENAI_API_VERSION=2024-02-15-preview
GPT4O_DEPLOYMENT_NAME=gpt-4o
GPT35_DEPLOYMENT_NAME=gpt-35-turbo
EMBEDDING_DEPLOYMENT_NAME=text-embedding-ada-002
DALLE_DEPLOYMENT_NAME=dall-e-3
# DALLE_DEFAULT_SIZE=1024x1024
# DALLE_DEFAULT_QUALITY=standard
# DALLE_DEFAULT_STYLE=vivid

COMPUTER_VISION_ENDPOINT=https://eastus.api.cognitive.microsoft.com/

SPEECH_SERVICES_ENDPOINT=https://aiservicesdemo-speech-dev-2ape77.cognitiveservices.azure.com/
SPEECH_SERVICES_REGION=eastus

LANGUAGE_SERVICES_ENDPOINT=https://eastus.api.cognitive.microsoft.com/

# セキュリティ上の理由により、APIキーは手動で設定してください:
# AZURE_OPENAI_API_KEY=your_openai_api_key
# COMPUTER_VISION_API_KEY=your_computer_vision_key
# SPEECH_SERVICES_API_KEY=your_speech_services_key
# LANGUAGE_SERVICES_API_KEY=your_language_servi

## 9. 次のステップとAPIキー設定

In [122]:
print("🎉 デプロイメント完了!")
print("\n📋 次のステップ:")
print("\n1. 📂 APIキーの取得:")
print(f"   - Azure Portal にアクセス: https://portal.azure.com")
print(f"   - リソースグループ '{RESOURCE_GROUP_NAME}' を開く")
print("   - 各AI Servicesリソースの 'キー' セクションからAPIキーをコピー")

print(f"\n2. 🔐 環境変数ファイルの更新:")
print(f"   - ファイル: {env_file_path}")
print("   - 各サービスのAPIキーを設定")

print("\n3. 🚀 デモの実行:")
print("   - demos/openai/01_GPT4o_Basic_Demo.ipynb を開く")
print("   - 環境変数ファイルを読み込んで実行")

print("\n4. 💰 コスト管理:")
print("   - 不要になったら必ずリソースを削除")
print(f"   - 削除コマンド: az group delete --name {RESOURCE_GROUP_NAME} --yes")

print("\n🔍 トラブルシューティング:")
print("   - デプロイメントエラーの確認: Azure Portal > デプロイメント")
print("   - リソース制限: 無料プランの場合は制限に注意")
print("   - リージョン制限: 一部のサービスは特定リージョンでのみ利用可能")

🎉 デプロイメント完了!

📋 次のステップ:

1. 📂 APIキーの取得:
   - Azure Portal にアクセス: https://portal.azure.com
   - リソースグループ 'rg-aiservices-demo' を開く
   - 各AI Servicesリソースの 'キー' セクションからAPIキーをコピー

2. 🔐 環境変数ファイルの更新:
   - ファイル: ../.env.dev
   - 各サービスのAPIキーを設定

3. 🚀 デモの実行:
   - demos/openai/01_GPT4o_Basic_Demo.ipynb を開く
   - 環境変数ファイルを読み込んで実行

4. 💰 コスト管理:
   - 不要になったら必ずリソースを削除
   - 削除コマンド: az group delete --name rg-aiservices-demo --yes

🔍 トラブルシューティング:
   - デプロイメントエラーの確認: Azure Portal > デプロイメント
   - リソース制限: 無料プランの場合は制限に注意
   - リージョン制限: 一部のサービスは特定リージョンでのみ利用可能


## 🛠️ トラブルシューティング用コマンド

問題が発生した場合は、以下のセルを実行して詳細を確認してください。

In [ ]:
# デプロイメント詳細の確認
detail_command = f"""az deployment group show \
+    --resource-group {RESOURCE_GROUP_NAME} \
+    --name {DEPLOYMENT_NAME} \
+    --output json"""

print("🔍 デプロイメント詳細情報:")
detail_result = run_command(detail_command, "デプロイメント詳細の取得")

In [ ]:
# エラーが発生した場合のデプロイメント操作履歴
operations_command = f"""az deployment operation group list \
+    --resource-group {RESOURCE_GROUP_NAME} \
+    --name {DEPLOYMENT_NAME} \
+    --output table"""

print("📋 デプロイメント操作履歴:")
operations_result = run_command(operations_command, "デプロイメント操作履歴の取得")

---

## 📚 参考リンク

- [Azure AI Services ドキュメント](https://docs.microsoft.com/azure/cognitive-services/)
- [Azure OpenAI Service](https://docs.microsoft.com/azure/cognitive-services/openai/)
- [Bicep ドキュメント](https://docs.microsoft.com/azure/azure-resource-manager/bicep/)
- [Azure CLI リファレンス](https://docs.microsoft.com/cli/azure/)

**🎯 このNotebookが正常に実行できれば、Azure AI Services Demoの環境が完全にセットアップされます！**